In [ ]:
import resources.workspace as ws
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as rnd
import scipy.linalg as sla
from mpl_tools.misc import nRowCol
from mpl_tools.place import freshfig

plt.ion();
rnd.seed(3000)

# Spatial statistics ("geostatistics")

Covariances between two (or a few) variables is very well,
but if you have not seen it before, the connection between covariances
and geophysical (spatial) fields may not be obvious.
The purpose of this tutorial is to familiarise you with random (spatial) fields
and their estimation.

In [ ]:
grid1D = np.linspace(0, 1, 61)
N = 15  # ensemble size

## Variograms
The "Variogram" of a field is essentially `1 - autocovariance`. Thus, it describes the spatial dependence of the field. The mean (1st moment) of a field is usually estimated and described/parametrized with trend lines/surfaces, while higher moments are usually not worth modelling.

In [ ]:
def variogram(dists, Range=1, kind="Gauss", nugget=0):
    """Compute variogram for distance points `dists`."""
    dists = dists / Range
    if kind == "Spheric":
        gamma = 1.5 * dists - .5 * dists**3
        gamma[dists >= 1] = 1
    elif kind == "Expo":
        dists *= 3  # by convention
        gamma = 1 - np.exp(-dists)
    else:  # "Gauss"
        dists *= 3  # by convention
        gamma = 1 - np.exp(-(dists)**2)
    # Include nugget (discontinuity at 0)
    gamma *= (1-nugget)
    gamma[dists != 0] += nugget
    return gamma

#### Plot

In [ ]:
@ws.interact(Range=(.01, 4), nugget=(0.0, 1, .1))
def plot_variogram(Range=1, nugget=0):
    fig, ax = freshfig("Variograms")  # fig = plt.figure(figsize=(6, 4))
    ax.grid()
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    for i, kind in enumerate(["Spheric", "Expo", "Gauss"]):
        gamma = variogram(grid1D, Range, kind, nugget=nugget)
        ax.plot(grid1D, gamma, lw=2, color=f"C{i}", label=kind)
        ax.legend(loc="upper left")

## Random fields (1D)

In order to apply the variogram we must be able to compute the distances between sets (A, B) of points. The following is a fairly efficient implementation.

In [ ]:
def dist_euclid(A, B):
    """Compute $ d_ij = \| a_i - b_j \|_2 $ for `a_i in A` and `b_j in B`."""
    diff = A[:, None, :] - B
    d2 = np.sum(diff**2, axis=-1)
    return np.sqrt(d2)

Gaussian random variables (vectors) are fully specified by their mean and covariance.
We can use the above variogram function to define the covariance (matrix) between points,
by first computing the distances between them.
Once in posession of a covariance matrix, we can use it to sample random variables
by multiplying its cholesky factor (square root) onto standard normal variables.

In [ ]:
def gaussian_fields(coords, **vg_params):
    """Gen. random (Gaussian) fields at `coords` (no structure/ordering required)."""
    dists = dist_euclid(coords, coords)
    covar = 1 - variogram(dists, **vg_params)
    cholL = sla.cholesky(covar).T
    fields = cholL @ rnd.randn(len(coords), N)
    return fields

#### Exc:
Use the plotting functionality below to
explain the effect of `Range` and `nugget`

In [ ]:
fig, ax = freshfig("1D random fields")
fields = gaussian_fields(grid1D[:, None], Range=1, kind="Gauss", nugget=1e-3)
ax.plot(grid1D, fields, lw=2);

## Random fields (2D)
The following sets up a 2d grid.

In [ ]:
grid2x, grid2y = np.meshgrid(grid1D, grid1D)
grid2x.shape

where `grid2y` has the same shape.

However, in the following we will "flatten" (a.k.a."(un)ravel", "vectorize", or "string out") this explicitly 2D grid of nodes into a simple list of points in 2D. Importantly, none of the following methods actually assume any structure to the list. So we could also work with a completely irregularly spaced set of points.

In [ ]:
grid2D = np.dstack([grid2x, grid2y]).reshape((-1, 2))
grid2D.shape

For example, `gaussian_fields` is immediately applicable also to this 2D case.

In [ ]:
vg_params = dict(Range=1, kind="Gauss", nugget=1e-4)
fields = gaussian_fields(grid2D, **vg_params)

Of course, for plotting purposes, we undo the flattening.

In [ ]:
def contour_plot(ax, field, cmap="nipy_spectral", levels=12, has_obs=True):
    field = field.reshape(grid2x.shape)  # undo flattening
    if has_obs:
        ax.plot(*obs_coo.T, "ko", ms=4)
        ax.plot(*obs_coo.T, "yo", ms=1)
    ax.set(aspect="equal", xticks=[0, 1], yticks=[0, 1])
    return ax.contourf(field, levels=levels, extent=(0, 1, 0, 1),
                       cmap=cmap, vmin=vmin, vmax=vmax)

# Fix the color scale for all subsequent `contour_plot`.
# Use `None` to re-compute the color scale for each subplot.
vmin = fields.min()
vmax = fields.max()

In [ ]:
fig, axs = freshfig(num="2D random fields", figsize=(5, 4),
                    nrows=3, ncols=4, sharex=True, sharey=True)

for ax, field in zip(axs.ravel(), fields.T):
    contour_plot(ax, field, has_obs=False)

## Estimation problem

For our estimation target we will use one of the above generated random fields.

In [ ]:
truth = fields.T[0]

For the observations, we pick some random grid locations for simplicity
(even though the methods work also with observations not on grid nodes).

In [ ]:
nObs = 10
obs_idx = rnd.randint(0, len(grid2D), nObs)
obs_coo = grid2D[obs_idx]
observations = truth[obs_idx]

## Spatial interpolant methods

In [ ]:
# Pre-compute re-used objects
dists_yy = dist_euclid(obs_coo, obs_coo)
dists_xy = dist_euclid(grid2D, obs_coo)

In [ ]:
estims = dict(Truth=truth)
vmin=truth.min()
vmax=truth.max()

The cells below contain snippets of different spatial interpolation methods
Complete the code snippets below.
Test your implementation by running the plotting code in the cell further below.

#### Exc: Nearest neighbour interpolation

In [ ]:
nearest_obs = np.argmin(dists_xy, 1)
estims["Nearest-n."] = observations[nearest_obs]

#### Exc: Inverse distance weighting

In [ ]:
expo = 3
with np.errstate(invalid='ignore', divide='ignore'):
    weights = 1/dists_xy**expo
    weights = weights / weights.sum(axis=1, keepdims=True)  # normalize

In [ ]:
# Apply weights
estims["Inv-dist."] = weights @ observations

In [ ]:
# Fix singularities
estims["Inv-dist."][obs_idx] = observations

#### Exc: Simple Kriging

In [ ]:
# Hint: use `sla.solve` or `sla.inv` (less recommended)
covar_yy = 1 - variogram(dists_yy, **vg_params)
cross_xy = 1 - variogram(dists_xy, **vg_params)
regression_coefficients = sla.solve(covar_yy, cross_xy.T).T

In [ ]:
estims["Kriging"] = regression_coefficients @ observations

### Plot truth, estimates, error

In [ ]:
fig, axs = freshfig(num="Estimation problem", figsize=(8, 4), squeeze=False,
                    nrows=2, ncols=len(estims), sharex=True, sharey=True)

for name, ax1, ax2 in zip(estims, *axs):
    ax1.set_title(name)
    c1 = contour_plot(ax1, estims[name])
    c2 = contour_plot(ax2, estims[name] - truth, cmap="RdBu")
fig.tight_layout()
fig.subplots_adjust(right=0.85)
cbar = fig.colorbar(c1, cax=fig.add_axes([0.9, 0.15, 0.03, 0.7]))

#### Exc: Try different values of `Range`.
- Run code to re-compute Kriging estimate.
- What does setting it to `0.1` cause? What about `100`?

In [ ]:
@ws.interact(Range=(.01, 40))
def plot_krieged(Range=1):
    vg_params['Range'] = Range
    covar_yy = 1 - variogram(dists_yy, **vg_params)
    cross_xy = 1 - variogram(dists_xy, **vg_params)
    regression_coefficients = sla.solve(covar_yy, cross_xy.T).T

    fig, ax = freshfig(num="Kriging estimates")
    c1 = contour_plot(ax, regression_coefficients @ observations)
    fig.colorbar(c1);

# Comments on Kriging
Generalizations
- Unknown mean (Ordinary Kriging)
- Co-Kriging (vector-valued fields)
- Trend surfaces (non-stationarity assumptions)

Kriging estimators can typically also be derived
as **Radial basis function (RBF) interpolation**,
or **Gaussian process regression** (GP) regression,
providing alternative interpretations.

- Kriging is derived by minimizing the variance of linear and unbiased estimators.
- RBF interpolation is derived by the explicit desire to fit
  N functions to N data points (observations).
- GP regression is derived by conditioning (applying Bayes rule)
  to the (supposedly) Gaussian distribution of the random field.